In [ ]:
# !pip install rltk

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import islice
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

### Load health violation and restaurant data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# All violations data with size, type and risk level
la_data = '/content/drive/MyDrive/DSCI 560 Project/Data/preprocessed_health_data.csv'
la_df = pd.read_csv(la_data)
la_df

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID
0,#1 CAFE,2080 CENTURY PARK E STE 108,LOS ANGELES,FA0056432,CA,90067,OW0000002,RESTAURANT (0-30) SEATS HIGH RISK,90.521739,"F014,F007,F033,F036,F027,F035,F034,F049,F040,F...",# 14. Food contact surfaces: clean and sanitiz...,restaurant,0-30,HIGH,0
1,#1 CHINESE FAST FOOD,8606 S VERMONT AVE,LOS ANGELES,FA0002813,CA,90044,OW0015192,RESTAURANT (0-30) SEATS HIGH RISK,93.096774,"F033,F043,F006,F007,F044,F034,F052,F035,F007,F...",# 33. Nonfood-contact surfaces clean and in go...,restaurant,0-30,HIGH,1
2,#1 DONUT,8509 S FIGUEROA ST,LOS ANGELES,FA0056433,CA,90003,OW0034724,RESTAURANT (0-30) SEATS MODERATE RISK,98.000000,"F035,F044",# 35. Equipment/Utensils - approved; installed...,restaurant,0-30,MODER,2
3,#1 DONUTS,8509 S FIGUEROA ST # 106,LOS ANGELES,FA0225526,CA,90003,OW0229782,RESTAURANT (0-30) SEATS MODERATE RISK,93.470588,"F008,F044,F037,F023,F036,F043,F044,F007,F014,F...",# 08. Time as a public health control; procedu...,restaurant,0-30,MODER,3
4,#2 MOON BBQ,478 N WESTERN AVE,LOS ANGELES,FA0002817,CA,90004,OW0024959,RESTAURANT (61-150) SEATS HIGH RISK,91.880952,"F013,F037,F033,F044,F033,F014,F046,F035,F033,F...","# 13. Food in good condition, safe and unadult...",restaurant,61-150,HIGH,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13298,ZUMA KITCHEN,1942 WESTWOOD BLVD,LOS ANGELES,FA0042262,CA,90025,OW0005436,RESTAURANT (61-150) SEATS HIGH RISK,90.409091,"F044,F027,F006,F033,F037,F035,F039,F052,F023,F...","# 44. Floors, walls and ceilings: properly bui...",restaurant,61-150,HIGH,13298
13299,ZUNIGA'S MARKET,3033 W 6TH ST STE 102,LOS ANGELES,FA0203029,CA,90020,OW0208291,"FOOD MKT RETAIL (25-1,999 SF) LOW RISK",98.000000,F006,# 06. Adequate handwashing facilities supplied...,food market,"25-1,999 SF",LOW,13299
13300,ZWEET CAFE,4682 EAGLE ROCK BLVD,LOS ANGELES,FA0177854,CA,90041,OW0183748,RESTAURANT (0-30) SEATS MODERATE RISK,95.125000,"F007,F044,F033,F030,F053,F037,F033,F044,F034,F...",# 07. Proper hot and cold holding temperatures...,restaurant,0-30,MODER,13300
13301,ZWONNY KITCHEN INC,857 S WESTERN AVE,LOS ANGELES,FA0160087,CA,90005,OW0125073,RESTAURANT (31-60) SEATS HIGH RISK,91.918919,"F026,F033,F052,F046,F035,F043,F033,F035,F023,F...","# 26. Approved thawing methods used, frozen fo...",restaurant,31-60,HIGH,13301


In [ ]:
# # add a column represent ID
# la_df['ID'] = range(0, 0+len(la_df))
# la_df.to_csv(la_data, index=False)

# la_df[la_df['FACILITY_ID']=='FA0242516']

### Restaurant data preprocessing

In [9]:
# read the restaurant data
# rest_data = '/content/drive/MyDrive/DSCI 560 Project/Data/restaurants_details_updated_0326_latest_update.csv'
rest_data = '/content/drive/MyDrive/DSCI 560 Project/Data/restaurants_details_updated_0409_latest_update.csv'
# rest_data = '/content/drive/MyDrive/DSCI 560 Project/Data/restaurant_all_test.csv'
# rest_df = pd.read_csv(rest_data).rename(columns={"Unnamed: 0": "restaurant_id"})

rest_df = pd.read_csv(rest_data)
rest_df.drop(columns='Unnamed: 0', inplace=True)
rest_df['restaurant_id'] = range(0, 0+len(rest_df))
rest_df.head()

,url,name,address,score,review_counts,price,category,open_time,amenities,restaurant_id
0,https://www.yelp.com/biz/tonchin-la-los-angeles,Tonchin LA,"5665 Melrose Ave Los Angeles, CA 90038",4.5,114 reviews,$$,Japanese,Mon\nClosed\nTue\n11:30 AM - 2:00 PM\n5:30 PM ...,Amenities and More\nHealth ScoreA\nTakes Reser...,0
1,https://www.yelp.com/biz/broken-mouth-lees-hom...,BROKEN MOUTH | Lee's Homestyle,"718 S Los Angeles St Los Angeles, CA 90014",4.8,(1.5k reviews),$$,"Hawaiian, Korean, Comfort Food",Mon\nClosed\nTue\n11:00 AM - 2:00 PM\n4:00 PM ...,Amenities and More\nHealth ScoreA\nOffers Deli...,1
2,https://www.yelp.com/biz/wood-los-angeles-8,Wood,"2861 Sunset Blvd Los Angeles, CA 90026",4.0,1017 reviews,$$,"Pizza, Italian, Wine Bars",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nTakes Reservations\nOffers...,2
3,https://www.yelp.com/biz/jail-joa-los-angeles,Jail Joa,"2500 W 8th St Ste 180 Los Angeles, CA 90057",4.6,(135 reviews),$$,"Gastropubs, Korean, Asian Fusion",Mon\n6:00 PM - 2:00 AM (Next day)\nTue\n6:00 P...,Amenities and More\nOffers Delivery\nOffers Ta...,3
4,https://www.yelp.com/biz/majordomo-los-angeles-2,Majordomo,"1725 Naud St Los Angeles, CA 90012",4.2,(1.3k reviews),$$$,"American (New), Asian Fusion, Korean",Mon\n5:00 PM - 10:00 PM\nTue\n5:00 PM - 10:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...,4


In [ ]:
# rest_df.rename(columns={"address": "new_address","zip": "zipcode" }, inplace=True)

In [10]:
rest_df

,url,name,address,score,review_counts,price,category,open_time,amenities,restaurant_id
0,https://www.yelp.com/biz/tonchin-la-los-angeles,Tonchin LA,"5665 Melrose Ave Los Angeles, CA 90038",4.5,114 reviews,$$,Japanese,Mon\nClosed\nTue\n11:30 AM - 2:00 PM\n5:30 PM ...,Amenities and More\nHealth ScoreA\nTakes Reser...,0
1,https://www.yelp.com/biz/broken-mouth-lees-hom...,BROKEN MOUTH | Lee's Homestyle,"718 S Los Angeles St Los Angeles, CA 90014",4.8,(1.5k reviews),$$,"Hawaiian, Korean, Comfort Food",Mon\nClosed\nTue\n11:00 AM - 2:00 PM\n4:00 PM ...,Amenities and More\nHealth ScoreA\nOffers Deli...,1
2,https://www.yelp.com/biz/wood-los-angeles-8,Wood,"2861 Sunset Blvd Los Angeles, CA 90026",4.0,1017 reviews,$$,"Pizza, Italian, Wine Bars",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nTakes Reservations\nOffers...,2
3,https://www.yelp.com/biz/jail-joa-los-angeles,Jail Joa,"2500 W 8th St Ste 180 Los Angeles, CA 90057",4.6,(135 reviews),$$,"Gastropubs, Korean, Asian Fusion",Mon\n6:00 PM - 2:00 AM (Next day)\nTue\n6:00 P...,Amenities and More\nOffers Delivery\nOffers Ta...,3
4,https://www.yelp.com/biz/majordomo-los-angeles-2,Majordomo,"1725 Naud St Los Angeles, CA 90012",4.2,(1.3k reviews),$$$,"American (New), Asian Fusion, Korean",Mon\n5:00 PM - 10:00 PM\nTue\n5:00 PM - 10:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...,4
...,...,...,...,...,...,...,...,...,...,...
5376,https://www.yelp.com/biz/kai-ramen-arcadia-arc...,Kai Ramen - Arcadia,"1130 S Baldwin Ave Ste 6 Arcadia, CA 91007",4.3,(469 reviews),$$,"Ramen, Noodles",Mon\n12:00 PM - 10:30 PM\nTue\n12:00 PM - 10:3...,Amenities and More\nHealth ScoreA\nOffers Deli...,5376
5377,https://www.yelp.com/biz/el-huarache-azteca-lo...,El Huarache Azteca,"5225 York Blvd Los Angeles, CA 90042",3.7,(996 reviews),$$,"Mexican, Breakfast & Brunch",Mon\n8:00 AM - 8:00 PM\nTue\n8:00 AM - 8:00 PM...,Amenities and More\nOffers Delivery\nNo Reserv...,5377
5378,https://www.yelp.com/biz/louisiana-fried-chick...,Louisiana Fried Chicken,"1515 N Lake Ave Pasadena, CA 91104",4.0,132 reviews,$,"Cajun/Creole, Chicken Wings",Mon\nClosed\nTue\n11:00 AM - 8:00 PM\nWed\n11:...,Amenities and More\nHealth Score96 out of 100\...,5378
5379,https://www.yelp.com/biz/cheeky-cheeky-bun-bun...,Cheeky Cheeky Bun Bun,"627 W Duarte Rd Arcadia, CA 91007",3.8,(19 reviews),NaN,Dim Sum,Mon\n11:00 AM - 9:30 PM\nTue\n11:00 AM - 9:30 ...,Amenities and More\nTakes Reservations\nOffers...,5379


Tranform open hours

In [11]:
# transform open time to hours per week
from datetime import datetime

def calculate_open_hours(x):
    format = '%I:%M %p'
    open_hours = 0
    if str(x) == 'nan':
        # print(f"{x} none")
        return None
    
    time_list = x.split('\n')
    # print(time_list)

    for t in time_list:
        try:
            # if int(t[0]):
            if 'AM' in t or 'PM' in t:
                # print(t)
                t_range = t.split('-')
                t_open = t_range[0].strip(' ')
                t_open = t_open.strip('\r')
                t_close = t_range[1].strip(' ')
                t_close = t_close.strip('\r')
                time_open = datetime.strptime(t_open, format)
                # print(time_open.time())
                time_close = datetime.strptime(t_close, format)
                # print(time_close.time())
                # print(time_close - time_open)
                open_h_day = time_close - time_open
                open_h_day = open_h_day.total_seconds()/3600  # covert to hours
                open_hours += open_h_day
        except:
            return None
    # print(open_hours)
    return open_hours

In [12]:
# add the open hour per week feature
rest_df['open_hours_week'] = rest_df['open_time'].apply(lambda x: calculate_open_hours(x))

Add the exact city to the restaurants

In [13]:
# split restaurant address to zipcode and state
def split_address(address):
    try:
        address_split = address.split(', ')
        if len(address_split) > 1:
            address = address_split[0]
            state_zip = address_split[1]
        else:
            address = address
            state_zip = None
        state_zip_split = state_zip.split(' ')
        if len(state_zip_split) > 1:
            state = state_zip_split[0]
            zip = state_zip_split[1]
        else:
            state = state_zip
            zip = None
        return address, state, zip
    except:
        # print(address)
        return None, None, None
    

In [14]:
# Split adress column into multiple columns use apply()
rest_df['new_address'] = rest_df["address"].apply(lambda x: split_address(x)[0])
rest_df['state'] = rest_df["address"].apply(lambda x: split_address(x)[1])
rest_df['zipcode'] = rest_df["address"].apply(lambda x: split_address(x)[2])
rest_df.head()

,url,name,address,score,review_counts,price,category,open_time,amenities,restaurant_id,open_hours_week,new_address,state,zipcode
0,https://www.yelp.com/biz/tonchin-la-los-angeles,Tonchin LA,"5665 Melrose Ave Los Angeles, CA 90038",4.5,114 reviews,$$,Japanese,Mon\nClosed\nTue\n11:30 AM - 2:00 PM\n5:30 PM ...,Amenities and More\nHealth ScoreA\nTakes Reser...,0,39.0,5665 Melrose Ave Los Angeles,CA,90038
1,https://www.yelp.com/biz/broken-mouth-lees-hom...,BROKEN MOUTH | Lee's Homestyle,"718 S Los Angeles St Los Angeles, CA 90014",4.8,(1.5k reviews),$$,"Hawaiian, Korean, Comfort Food",Mon\nClosed\nTue\n11:00 AM - 2:00 PM\n4:00 PM ...,Amenities and More\nHealth ScoreA\nOffers Deli...,1,30.0,718 S Los Angeles St Los Angeles,CA,90014
2,https://www.yelp.com/biz/wood-los-angeles-8,Wood,"2861 Sunset Blvd Los Angeles, CA 90026",4.0,1017 reviews,$$,"Pizza, Italian, Wine Bars",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nTakes Reservations\nOffers...,2,84.0,2861 Sunset Blvd Los Angeles,CA,90026
3,https://www.yelp.com/biz/jail-joa-los-angeles,Jail Joa,"2500 W 8th St Ste 180 Los Angeles, CA 90057",4.6,(135 reviews),$$,"Gastropubs, Korean, Asian Fusion",Mon\n6:00 PM - 2:00 AM (Next day)\nTue\n6:00 P...,Amenities and More\nOffers Delivery\nOffers Ta...,3,NaN,2500 W 8th St Ste 180 Los Angeles,CA,90057
4,https://www.yelp.com/biz/majordomo-los-angeles-2,Majordomo,"1725 Naud St Los Angeles, CA 90012",4.2,(1.3k reviews),$$$,"American (New), Asian Fusion, Korean",Mon\n5:00 PM - 10:00 PM\nTue\n5:00 PM - 10:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...,4,36.0,1725 Naud St Los Angeles,CA,90012


In [15]:
zipcode_df = pd.read_csv('/content/drive/MyDrive/DSCI 560 Project/Data/zipcode.csv')
zipcode_df


,zipcode,city
0,93510,Acton
1,91301,Agoura Hills
2,91376,Agoura Hills
3,91801,Alhambra
4,91802,Alhambra
...,...,...
523,91364,Woodland Hills
524,91365,Woodland Hills
525,91367,Woodland Hills
526,91371,Woodland Hills


In [16]:
zipcode_dict = dict(zip([str(x) for x in zipcode_df['zipcode'].values], zipcode_df['city'].values))
# zipcode_dict

In [17]:
la_df['FACILITY_ZIP'].map(zipcode_dict).unique()

array(['Los Angeles', nan, 'Compton', 'West Hollywood', 'Northridge',
       'Venice', 'Culver City', 'Playa Vista', 'West Hills',
       'North Hollywood', 'Huntington Park', 'Studio City', 'Burbank',
       'Beverly Hills', 'Pacific Palisades', 'Porter Ranch',
       'Bell Gardens', 'Marina Del Rey', 'San Fernando', 'Paramount',
       'Woodland Hills', 'Arcadia'], dtype=object)

In [ ]:
rest_df['city'] = rest_df['zipcode'].map(zipcode_dict)

In [ ]:
rest_df.head()

,url,name,address,score,review_counts,price,category,open_time,amenities,restaurant_id,new_address,state,zipcode,city
0,https://www.yelp.com/biz/tonchin-la-los-angeles,Tonchin LA,"5665 Melrose Ave Los Angeles, CA 90038",4.5,114 reviews,$$,Japanese,Mon\nClosed\nTue\n11:30 AM - 2:00 PM\n5:30 PM ...,Amenities and More\nHealth ScoreA\nTakes Reser...,0,5665 Melrose Ave Los Angeles,CA,90038,Los Angeles
1,https://www.yelp.com/biz/broken-mouth-lees-hom...,BROKEN MOUTH | Lee's Homestyle,"718 S Los Angeles St Los Angeles, CA 90014",4.8,(1.5k reviews),$$,"Hawaiian, Korean, Comfort Food",Mon\nClosed\nTue\n11:00 AM - 2:00 PM\n4:00 PM ...,Amenities and More\nHealth ScoreA\nOffers Deli...,1,718 S Los Angeles St Los Angeles,CA,90014,Los Angeles
2,https://www.yelp.com/biz/wood-los-angeles-8,Wood,"2861 Sunset Blvd Los Angeles, CA 90026",4.0,1017 reviews,$$,"Pizza, Italian, Wine Bars",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nTakes Reservations\nOffers...,2,2861 Sunset Blvd Los Angeles,CA,90026,Los Angeles
3,https://www.yelp.com/biz/jail-joa-los-angeles,Jail Joa,"2500 W 8th St Ste 180 Los Angeles, CA 90057",4.6,(135 reviews),$$,"Gastropubs, Korean, Asian Fusion",Mon\n6:00 PM - 2:00 AM (Next day)\nTue\n6:00 P...,Amenities and More\nOffers Delivery\nOffers Ta...,3,2500 W 8th St Ste 180 Los Angeles,CA,90057,Los Angeles
4,https://www.yelp.com/biz/majordomo-los-angeles-2,Majordomo,"1725 Naud St Los Angeles, CA 90012",4.2,(1.3k reviews),$$$,"American (New), Asian Fusion, Korean",Mon\n5:00 PM - 10:00 PM\nTue\n5:00 PM - 10:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...,4,1725 Naud St Los Angeles,CA,90012,Los Angeles


In [ ]:
rest_df = rest_df[~rest_df.name.isnull()]

In [ ]:
rest_df = rest_df[~rest_df.address.isnull()]

In [ ]:
rest_df = rest_df[rest_df.address.str.contains('CA')]

In [ ]:
rest_df[rest_df.zipcode.isnull()]

,url,name,address,score,review_counts,price,category,open_time,amenities,restaurant_id,new_address,state,zipcode,city
2603,https://www.yelp.com/biz/7-machos-mariscos-pac...,7 Machos Mariscos,"Pacoima, CA",5.0,16 reviews,NaN,"Food Trucks, Seafood, Mexican",Mon\n10:00 AM - 6:00 PM\nTue\n10:00 AM - 6:00 ...,Amenities and More\nMany Vegetarian Options,3626,Pacoima,CA,None,NaN
2894,https://www.yelp.com/biz/delta-pride-fish-adel...,Delta Pride Fish,"16900 Lakewood Blvd Ste 203 Adelanto, CA",NaN,NaN,NaN,Seafood,Mon\nClosed\nTue\n11:00 AM - 6:00 PM\nWed\n11:...,NaN,3919,16900 Lakewood Blvd Ste 203 Adelanto,CA,None,NaN


In [ ]:
rest_df.isnull().sum()

url                 0
name                0
address             0
score             387
review_counts     387
price            1102
category            0
open_time         741
amenities         396
restaurant_id       0
new_address         0
state               0
zipcode             2
city               18
dtype: int64

In [ ]:
rest_df.city.unique()

array(['Los Angeles', 'Culver City', 'North Hollywood', nan,
       'Universal City', 'Studio City', 'Gardena', 'Long Beach',
       'Compton', 'Carson', 'Lakewood', 'Torrance', 'Lynwood',
       'Signal Hill', 'South Gate', 'Huntington Park', 'Redondo Beach',
       'Harbor City', 'Downey', 'Bellflower', 'Bell Gardens',
       'West Hollywood', 'Van Nuys', 'Reseda', 'North Hills', 'Encino',
       'Northridge', 'Panorama City', 'Woodland Hills', 'Tarzana',
       'Winnetka', 'Canoga Park', 'Chatsworth', 'Porter Ranch', 'Pacoima',
       'Granada Hills', 'Sherman Oaks', 'West Hills', 'Venice',
       'Playa Del Rey', 'Marina Del Rey', 'Santa Monica', 'Playa Vista',
       'Inglewood', 'Calabasas', 'Valley Village', 'Burbank', 'Sunland',
       'Sun Valley', 'Glendale', 'Maywood', 'Bell', 'Tujunga',
       'La Crescenta', 'Beverly Hills', 'Pacific Palisades',
       'Pico Rivera', 'Montebello', 'San Fernando', 'Mission Hills',
       'Sylmar', 'Paramount', 'Agoura Hills', 'Pasadena', 'S

In [ ]:
rest_df.reset_index(inplace=True, drop=True)

In [ ]:
rest_df.drop(columns=['index'],inplace=True)

In [ ]:
rest_df.head()

In [ ]:
rest_df.to_csv('rest_data.csv', index=False)

In [ ]:
print(f'Altogether we have {len(la_df)} health records and {len(rest_df)} restaurant data.')

Altogether we have 13303 health records and 3355 restaurant data.


### Construct RLTK Datasets

In [ ]:
# !pip install rltk

In [ ]:
import rltk

# construct a tokenizer
tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()

In [ ]:
la_df

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID
0,#1 CAFE,2080 CENTURY PARK E STE 108,LOS ANGELES,FA0056432,CA,90067,OW0000002,RESTAURANT (0-30) SEATS HIGH RISK,90.521739,"F014,F007,F033,F036,F027,F035,F034,F049,F040,F...",# 14. Food contact surfaces: clean and sanitiz...,restaurant,0-30,HIGH,0
1,#1 CHINESE FAST FOOD,8606 S VERMONT AVE,LOS ANGELES,FA0002813,CA,90044,OW0015192,RESTAURANT (0-30) SEATS HIGH RISK,93.096774,"F033,F043,F006,F007,F044,F034,F052,F035,F007,F...",# 33. Nonfood-contact surfaces clean and in go...,restaurant,0-30,HIGH,1
2,#1 DONUT,8509 S FIGUEROA ST,LOS ANGELES,FA0056433,CA,90003,OW0034724,RESTAURANT (0-30) SEATS MODERATE RISK,98.000000,"F035,F044",# 35. Equipment/Utensils - approved; installed...,restaurant,0-30,MODER,2
3,#1 DONUTS,8509 S FIGUEROA ST # 106,LOS ANGELES,FA0225526,CA,90003,OW0229782,RESTAURANT (0-30) SEATS MODERATE RISK,93.470588,"F008,F044,F037,F023,F036,F043,F044,F007,F014,F...",# 08. Time as a public health control; procedu...,restaurant,0-30,MODER,3
4,#2 MOON BBQ,478 N WESTERN AVE,LOS ANGELES,FA0002817,CA,90004,OW0024959,RESTAURANT (61-150) SEATS HIGH RISK,91.880952,"F013,F037,F033,F044,F033,F014,F046,F035,F033,F...","# 13. Food in good condition, safe and unadult...",restaurant,61-150,HIGH,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13298,ZUMA KITCHEN,1942 WESTWOOD BLVD,LOS ANGELES,FA0042262,CA,90025,OW0005436,RESTAURANT (61-150) SEATS HIGH RISK,90.409091,"F044,F027,F006,F033,F037,F035,F039,F052,F023,F...","# 44. Floors, walls and ceilings: properly bui...",restaurant,61-150,HIGH,13298
13299,ZUNIGA'S MARKET,3033 W 6TH ST STE 102,LOS ANGELES,FA0203029,CA,90020,OW0208291,"FOOD MKT RETAIL (25-1,999 SF) LOW RISK",98.000000,F006,# 06. Adequate handwashing facilities supplied...,food market,"25-1,999 SF",LOW,13299
13300,ZWEET CAFE,4682 EAGLE ROCK BLVD,LOS ANGELES,FA0177854,CA,90041,OW0183748,RESTAURANT (0-30) SEATS MODERATE RISK,95.125000,"F007,F044,F033,F030,F053,F037,F033,F044,F034,F...",# 07. Proper hot and cold holding temperatures...,restaurant,0-30,MODER,13300
13301,ZWONNY KITCHEN INC,857 S WESTERN AVE,LOS ANGELES,FA0160087,CA,90005,OW0125073,RESTAURANT (31-60) SEATS HIGH RISK,91.918919,"F026,F033,F052,F046,F035,F043,F033,F035,F023,F...","# 26. Approved thawing methods used, frozen fo...",restaurant,31-60,HIGH,13301


In [ ]:
# la_df.to_csv('final_data.csv',index=False)

In [ ]:
# construct the attributes for health data
class HeathData(rltk.Record):
    # def __init__(self, raw_object):
    #     super().__init__(raw_object)
    #     self.name = ''

    @property
    def id(self):
        return self.raw_object['ID']

    @property
    def fid(self):
        return self.raw_object['FACILITY_ID']

    @property
    def name(self):
        return self.raw_object['FACILITY_NAME']

    @property
    def address(self):
        return self.raw_object['FACILITY_ADDRESS']

    @property
    def zipcode(self):
        return self.raw_object['FACILITY_ZIP']  
    
    @property
    def onwer_id(self):
        return self.raw_object['OWNER_ID']

In [ ]:
# construct the attributes for restaurant data
class RestData(rltk.Record):
    @property
    def id(self):
        return self.raw_object['restaurant_id']

    @property
    def name(self):
        return self.raw_object['name']

    @property
    def address(self):
        return self.raw_object['new_address']
    
    @property
    def zipcode(self):
        return self.raw_object['zipcode']  

In [ ]:
rest_data = '/content/rest_data.csv'
# construct datasets
ds1 = rltk.Dataset(rltk.CSVReader(open(la_data,encoding='utf-8')), record_class=HeathData)
ds2 = rltk.Dataset(rltk.CSVReader(open(rest_data,encoding='utf-8')), record_class=RestData)

In [ ]:
print(ds1.generate_dataframe().head())
print(ds2.generate_dataframe().head())

  id        fid                  name                      address zipcode  \
0  0  FA0056432               #1 CAFE  2080 CENTURY PARK E STE 108   90067   
1  1  FA0002813  #1 CHINESE FAST FOOD           8606 S VERMONT AVE   90044   
2  2  FA0056433              #1 DONUT           8509 S FIGUEROA ST   90003   
3  3  FA0225526             #1 DONUTS     8509 S FIGUEROA ST # 106   90003   
4  4  FA0002817           #2 MOON BBQ            478 N WESTERN AVE   90004   

    onwer_id  
0  OW0000002  
1  OW0015192  
2  OW0034724  
3  OW0229782  
4  OW0024959  
  id                            name                            address  \
0  0                      Tonchin LA       5665 Melrose Ave Los Angeles   
1  1  BROKEN MOUTH | Lee's Homestyle   718 S Los Angeles St Los Angeles   
2  2                            Wood       2861 Sunset Blvd Los Angeles   
3  3                        Jail Joa  2500 W 8th St Ste 180 Los Angeles   
4  4                       Majordomo           1725 Naud St Los An

In [ ]:
ds1.generate_dataframe().sample(5)

,id,fid,name,address,zipcode,onwer_id
1594,1594,FA0251731,BUFFALO WILD WINGS,3150 WILSHIRE BLVD,90010-1331,OW0245628
10019,10019,FA0225689,SABORES OAXAQENOS RESTAURANT,5028 MELROSE AVE,90038,OW0229935
4769,4769,FA0244632,"HAIRUN TRADING CO, INC",839 N BROADWAY # A,90012-2309,OW0239819
10368,10368,FA0047629,SHIN SUN MEAT,861 S WESTERN AVE,90005,OW0018035
8414,8414,FA0026579,NORTHGATE MARKET,2323 W OLYMPIC BLVD,90006,OW0026268


In [ ]:
ds2.generate_dataframe().sample(5)

,id,name,address,zipcode
934,940,On Tilt Pinball,7128 Van Nuys Blvd Ayce Gogi Los Angeles,91405
2591,3614,Raspado Xpress,13796 Foothill Blvd Los Angeles,91342
2372,3390,El Aji Picante,20905 Lassen St Chatsworth,91311
295,298,3355 Asian Grill,1955 Torrance Blvd Torrance,90501
620,624,Ike's Love & Sandwiches - COMING SOON,4150 McGowen St Ste 12 Long Beach,90846


In [ ]:
print(f"constructed datasets: \nLength of health data = {len(ds1.generate_dataframe())} \nLength of restaurant data = {len(ds2.generate_dataframe())}")

constructed datasets: 
Length of health data = 13303 
Length of restaurant data = 3355


In [ ]:
# 13303*7119

94704057

### Blocking

If we calculate on the whole datasets, there will be too many pairs -- extremely slow, so calculate on the blocked datasets instead.

In [ ]:
def generate_pairs(k_v_dic, pairs):
    if len(list(k_v_dic.keys()))>1:
        for health_r in k_v_dic['HeathData']:
            for rest_r in k_v_dic['RestData']:
                pairs.append([health_r,rest_r])

In [ ]:
def get_pairs(key):
    pairs_blocked = []
    for idx, content in enumerate(key):
        k_v_dic = {} # store the key,value pair of each block
        records = content[1]
        # print(records)
        for r in records:
            k = r[0]
            v = r[1]
            if k not in k_v_dic:
                k_v_dic[k] = [v]
            else:
                k_v_dic[k].append(v)
        # print(k_v_dic)
        generate_pairs(k_v_dic, pairs_blocked)
    
    return pairs_blocked

In [ ]:
# # small sample size, may not need to block
print('--- block on zipcode ---')

bg_zipcode = rltk.HashBlockGenerator()
block_zipcode= bg_zipcode.generate(
    bg_zipcode.block(ds1, property_='zipcode'),
    bg_zipcode.block(ds2, property_='zipcode')
)

# show part of the content
for idx, content in enumerate(block_zipcode.key_set_adapter):
  if idx==5: break
  print(content)

pairs_blocked_zipcode = get_pairs(block_zipcode.key_set_adapter)

print('Some of the newly generated pairs are:', str(pairs_blocked_zipcode[:10]))

--- block on zipcode ---
('90067', {('HeathData', '1530'), ('RestData', '3379'), ('HeathData', '3374'), ('HeathData', '8850'), ('HeathData', '7507'), ('HeathData', '9670'), ('HeathData', '9887'), ('HeathData', '11837'), ('HeathData', '11436'), ('HeathData', '8107'), ('HeathData', '2618'), ('HeathData', '4963'), ('HeathData', '12333'), ('HeathData', '11344'), ('HeathData', '1431'), ('HeathData', '12320'), ('HeathData', '8485'), ('HeathData', '5321'), ('HeathData', '1229'), ('HeathData', '10734'), ('HeathData', '8337'), ('HeathData', '2226'), ('HeathData', '3899'), ('HeathData', '13052'), ('HeathData', '21'), ('HeathData', '10522'), ('HeathData', '2549'), ('HeathData', '9610'), ('HeathData', '11075'), ('HeathData', '10521'), ('HeathData', '5198'), ('HeathData', '11757'), ('HeathData', '7843'), ('HeathData', '9124'), ('HeathData', '341'), ('HeathData', '8332'), ('HeathData', '1211'), ('HeathData', '4778'), ('HeathData', '1230'), ('HeathData', '2104'), ('HeathData', '10648'), ('HeathData',

In [ ]:
# Now we calculate the total pairs after blocking with multiple attributes.
# total_pair_block= 0
# for idx, content in enumerate(block_zipcode.key_set_adapter):
#     # print(idx, content)
#     health_num = 0
#     rest_num = 0
#     print(content)
#     for k,v in content[1]:
#         if k=='HealthData':
#             health_num += 1
#         elif k=='RestData':
#             rest_num += 1
#     total_pair_block += health_num * rest_num
total_pair_block_zipcode = len(pairs_blocked_zipcode)
print('After blocking with zipcode, there are %d pairs in the whole dataset.'%total_pair_block_zipcode)

After blocking with zipcode, there are 241872 pairs in the whole dataset.


Then calculate the reduction ratio for the blocking.

In [ ]:
size_health_data = len(ds1.generate_dataframe()) # length of health records
size_rest_data = len(ds2.generate_dataframe()) # length of restaurant records
total_pairs = size_health_data * size_rest_data

In [ ]:
print(f'Total pair size is {total_pairs}')
print(f'Total blocked pair is {total_pair_block_zipcode}')
reduction_block_zipcode = total_pair_block_zipcode / total_pairs
print(f'The proportion of new candidates if we block zipcode is {reduction_block_zipcode}.')

Total pair size is 44631565
Total blocked pair is 241872
The proportion of new candidates if we block zipcode is 0.0054193035803248215.


We can also try to block multiple attributes.

In [ ]:
print('--- block on address[:4] + zipcode ---')
bg_multi_zipcode = rltk.HashBlockGenerator()
block_multi_zipcode = bg_multi_zipcode.generate(
            bg_multi_zipcode.block(ds1, function_=lambda r: r.zipcode[:5], property_='zipcode'),
            bg_multi_zipcode.block(ds2, function_=lambda r: r.zipcode[:5], property_='zipcode'))

bg_multi_address = rltk.HashBlockGenerator()
block_multi_address = bg_multi_address.generate(
            bg_multi_address.block(ds1, function_=lambda r: r.address[:4],base_on=block_multi_zipcode),
            bg_multi_address.block(ds2, function_=lambda r: r.address[:4],base_on=block_multi_zipcode))

# show part of the content
for idx, content in enumerate(block_multi_address.key_set_adapter):
  if idx==5: break
  print(content)

# pairs_blocked_multi = []
pairs_blocked_multi = get_pairs(block_multi_address.key_set_adapter)
print('Some of the newly generated pairs are:', str(pairs_blocked_multi[:10]))

--- block on address[:4] + zipcode ---
('90067-1010', {('HeathData', '1719'), ('HeathData', '1530'), ('HeathData', '2709'), ('HeathData', '1431')})
('90067-1025', {('HeathData', '3374'), ('HeathData', '8850'), ('HeathData', '7507'), ('HeathData', '9887'), ('HeathData', '11436'), ('HeathData', '2618'), ('HeathData', '8485'), ('HeathData', '1229'), ('HeathData', '8337'), ('HeathData', '2226'), ('HeathData', '3899'), ('HeathData', '10522'), ('HeathData', '2549'), ('HeathData', '9610'), ('HeathData', '11075'), ('HeathData', '10521'), ('HeathData', '11757'), ('HeathData', '7843'), ('HeathData', '341'), ('HeathData', '8332'), ('HeathData', '1211'), ('HeathData', '4778'), ('HeathData', '1230'), ('HeathData', '10648'), ('HeathData', '10316'), ('RestData', '3373'), ('HeathData', '7768'), ('HeathData', '796'), ('HeathData', '7675'), ('HeathData', '4682'), ('HeathData', '2652'), ('HeathData', '6531'), ('HeathData', '1524'), ('HeathData', '1208'), ('HeathData', '4007'), ('HeathData', '5687'), ('He

In [ ]:
total_pair_block_multi = len(pairs_blocked_multi)
print('After blocking with address[:4] + zipcode, there are %d pairs in the whole dataset.'%total_pair_block_multi)

After blocking with address[:4] + zipcode, there are 2833 pairs in the whole dataset.


In [ ]:
print(f'Total pair size is {total_pairs}')
print(f'Total blocked pair is {total_pair_block_multi}')
reduction_block_zipcode = total_pair_block_multi / total_pairs
print(f'The proportion of new candidates if we block multiple attributes is {reduction_block_zipcode}.')

Total pair size is 44631565
Total blocked pair is 2833
The proportion of new candidates if we block multiple attributes is 6.347525568507401e-05.


### Load the ground truth and calculate the precision/recall of the blocking part.

In [ ]:
gt_df = pd.read_csv('/content/drive/MyDrive/DSCI 560 Project/Data/ground_truth.csv')
gt_df.head(2)

,RECORD_ID,FACILITY_ID,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,OWNER_NAME,PE_DESCRIPTION,PROGRAM_NAME,PROGRAM_STATUS,SCORE,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities
0,PR0190697,FA0242516,NERANO,9960 S SANTA MONICA BLVD,BEVERLY HILLS,CA,90212,OW0238121,BTBH . LLC,RESTAURANT (31-60) SEATS HIGH RISK,NERANO,ACTIVE,92.7,5,https://www.yelp.com/biz/nerano-beverly-hills?...,Nerano,"9960 S Santa Monica Blvd Beverly Hills, CA 90212",4.0,237 reviews,$$$,"Italian, Cocktail Bars",Mon\r\n11:30 AM - 2:30 PM\r\n5:00 PM - 9:00 PM...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
1,NaN,FA0177063,PASTA SISTERS,3343 W PICO BLVD,LOS ANGELES,CA,90019-4530,OW0183037,"LA CUCINA DA RE, LLC",RESTAURANT (0-30) SEATS HIGH RISK,PASTA SISTERS,ACTIVE,98.0,26,https://www.yelp.com/biz/pasta-sisters-los-ang...,Pasta Sisters,"3343 W Pico Blvd Los Angeles, CA 90019",4.5,1910 reviews,$$,"Delis, Italian, Sandwiches",Mon\r\n11:30 AM - 8:30 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nHealth ScoreA\r\nOffers ...


Map the facility id in the ground truth to the ID we manually created.

In [ ]:
facility_id_map = dict(zip(la_df['FACILITY_ID'], la_df['ID']))

In [ ]:
gt_df['ID'] = gt_df['FACILITY_ID'].map(facility_id_map)

In [ ]:
gt_df = gt_df[~gt_df['ID'].isnull()]
# gt_df

In [ ]:
gt_length = len(gt_df)
print(f"we got {gt_length} ground truth pairs")

we got 49 ground truth pairs


In [ ]:
# groud truth pairs
gt_dict = dict(zip([int(x) for x in gt_df['ID'].tolist()],gt_df['restaurant_id'].tolist()))
dict(islice(gt_dict.items(), 0, 5))

{8930: 26, 944: 71, 11533: 207, 1127: 245, 10575: 268}

In [ ]:
# precision = TP / (TP + FP)
# actually true / predicted true

# pairs_blocked_multi: ('RestData', '3925')
tp = 0
predicted_pair_list = []
# for p in pairs_blocked_multi:
    # id1 = int(p[0])
    # id2 = int(p[1])
#     if (id1, id2) in gt_dict.items():
#         tp += 1

for p in pairs_blocked_multi:
    id1 = int(p[0])
    id2 = int(p[1])
    if id1 in gt_dict.keys():
        predicted_pair_list.append((id1, id2))
    # elif id2 in gt_dict.values():
    #     if (id1, id2) not in predicted_pair_list:
    #         predicted_pair_list.append((id1, id2))

for p in predicted_pair_list:
    if p in gt_dict.items():
        tp += 1


precision = tp / len(predicted_pair_list)
print(f"blocking precision = {precision}")

blocking precision = 0.34265734265734266


DSCI558: The reduction ratio should be under 0.1 while pair completeness is over 0.7(RECALL). So the low precision makes sense.

In [ ]:
# recall = TP / (TP + FN)
# how many true we predicted / number of actual true
tp = 0
for pair in gt_dict.items():
    blocked_pair = [str(pair[0]), str(pair[1])]
    # print(pair)
    if blocked_pair in pairs_blocked_multi:
        tp += 1
    # else:
    #     print(blocked_pair)

# actually we are checking 49 pairs
recall = tp / gt_length
print(f"blocking recall = {recall}")

blocking recall = 0.0


In [ ]:
# ['8930', '26']
# ['944', '71']
# ['11533', '207']
# ['1127', '245']
# ['10575', '268']
la_df[la_df['ID']==8930]

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID
8930,PASTA SISTERS,3343 W PICO BLVD,LOS ANGELES,FA0177063,CA,90019-4530,OW0183037,RESTAURANT (0-30) SEATS HIGH RISK,96.238095,"F036,F007,F036,F025,F040,F035,F044,F007,F035,F...","# 36. Equipment, utensils and linens: storage ...",restaurant,0-30,HIGH,8930


In [ ]:
rest_df[rest_df['restaurant_id']==26]

,Unnamed: 0,url,name,address,score,review_counts,price,category,open_time,amenities,restaurant_id,open_hours_week,new_address,state,zipcode,city
26,26,https://www.yelp.com/biz/pasta-sisters-los-ang...,Pasta Sisters,"3343 W Pico Blvd Los Angeles, CA 90019",4.5,1910 reviews,$$,"Delis, Italian, Sandwiches",Mon\r\n11:30 AM - 8:30 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nHealth ScoreA\r\nOffers ...,26,54.0,3343 W Pico Blvd Los Angeles,CA,90019,Los Angeles


In [ ]:
f1_score = 2 * precision * recall / (precision + recall)
print(f"blocking f1 score = {f1_score}")

blocking f1 score = 0.5104166666666666


### Entity Linking

In [ ]:
# levenshtein distance
# attention: should be an inverse because this is the distance

# jaro winkler similarity for name
def jw_similarity_name(r1,r2):
  s1 = r1.name.lower()
  s2 = r2.name.lower()
  jw_sim = rltk.jaro_winkler_similarity(s1,s2)
  # print(jw_sim)
  return jw_sim

def lev_distance(r1, r2):
  s1 = r1.name.lower()
  s2 = r2.name.lower()
  lev_dist = rltk.levenshtein_distance(s1, s2)
  if lev_dist > min(len(s1), len(s2)) / 3:
    return False, lev_dist
  return True, lev_dist

# zipcode
def zipcode_similarity(r1,r2):
  zipcode1 = r1.zipcode
  zipcode2 = r2.zipcode
  if zipcode1==zipcode1:
    return 1
  else:
    return 0

# jaro winkler similarity for address
def jw_similarity_address(r1,r2):
  s1 = r1.address.lower()
  s2 = r2.address.lower()
  jw_sim = rltk.jaro_winkler_similarity(s1,s2)
  # print(jw_sim)
  return jw_sim


In [ ]:
# define the treshold
my_thresh_1 = 0.7

# weighted entity linking score function
def rule_based_method(r1,r2):
  jw_sim_name = jw_similarity_name(r1,r2)
  zipcode_sim = zipcode_similarity(r1,r2)
  jw_sim_address = jw_similarity_address(r1,r2)

  score = 0.5 * jw_sim_name + 0.2 * zipcode_sim + 0.3 * jw_sim_address  # because we have already blocked the first 4 characters of the address, so here we can have lower weight
#   score = 0.4 * jw_sim_name + 0.2 * zipcode_sim + 0.4 * jw_sim_address
  return score > my_thresh_1, score

### Record Likage

In [ ]:
# valid_prediction_pair = []
# for p in pairs_blocked_zipcode:
#     id1 = p[0]
#     id2 = p[1]
#     r1 = ds1.get_record(id1)
#     r2  = ds2.get_record(id2)

#     result, confidence = rule_based_method(r1, r2)
#     if result:
#         valid_prediction_pair.append(p)

# only keep the pair with highest confidence
res_dict = {}
valid_prediction_pair_multi = []

for p in pairs_blocked_multi:
    id1 = p[0]
    id2 = p[1]
    res_dict[id1] = (id2, 0)

for p in pairs_blocked_multi:
    id1 = p[0]
    id2 = p[1]
    r1 = ds1.get_record(id1)
    r2 = ds2.get_record(id2)

    result, confidence = rule_based_method(r1, r2)

    if result:
        # if confidence >= res_dict[id1][-1]:
        #     res_dict[id1] = (id2, confidence)

            valid_prediction_pair_multi.append(p)
        # res_dict[(id1, id2)] = confidence
        # if confidence>=res_dict[(id1, id2)]:
        #     res_dict[(id1, id2)] = confidence

# for pair in res_dict:
#     valid_prediction_pair_multi.append(p)

In [ ]:
# dict(islice(res_dict.items(), 0, 10))
valid_prediction_pair_multi[:10]

[['7507', '3362'],
 ['9887', '3373'],
 ['9887', '3362'],
 ['9887', '3344'],
 ['11436', '3362'],
 ['2618', '3373'],
 ['2618', '3362'],
 ['2618', '3344'],
 ['8485', '3362'],
 ['8337', '3373']]

In [ ]:
la_df[la_df['ID']==12257]

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID
12257,TOSAI,10250 SANTA MONICA BLVD # FC2,LOS ANGELES,FA0256355,CA,90067,OW0017154,RESTAURANT (0-30) SEATS HIGH RISK,94.666667,"F005,F035,F007,F049,F035,F036",# 05. Hands clean and properly washed; gloves ...,restaurant,0-30,HIGH,12257


In [ ]:
rest_df[rest_df['restaurant_id']==3362]

,url,name,address,score,review_counts,price,category,open_time,amenities,restaurant_id,new_address,state,zipcode,city
3362,https://www.yelp.com/biz/tosai-sushi-los-angeles,Tosai Sushi,"10250 Santa Monica Blvd Los Angeles, CA 90067",3.5,44 reviews,NaN,Sushi Bars,Mon\n10:00 AM - 9:00 PM\nTue\n10:00 AM - 9:00 ...,Amenities and More\nHealth ScoreA\nOffers Deli...,3362,10250 Santa Monica Blvd Los Angeles,CA,90067,Los Angeles


In [ ]:
# # # only keep the pair with highest confidence
# for key, value in res_dict.items():
#     if value[-1]:
#         pair = (key, value[0])
#         valid_prediction_pair_multi.append(pair)

In [ ]:
len(valid_prediction_pair_multi)  # duplicates because there seems to be in the health_data, 2944

1549

In [ ]:
# valid_prediction_pair_multi

### Out put the result and Calculate the precision and recall of the prediction

In [ ]:
# output the result
import csv
with open('linked_restaurant_predictions_multi.csv','w',newline='') as f:
    writer = csv.writer(f)
    header = ['health.ID', 'restaurant.ID'] # health id is the ID in violation data
    writer.writerow(header)
    for p in valid_prediction_pair_multi:
        writer.writerow(p)

In [ ]:
valid_prediction_pair_multi[:2]

[['7507', '3362'], ['9887', '3373']]

In [ ]:
dict(islice(gt_dict.items(), 0, 5))

{8930: 26, 944: 71, 11533: 207, 1127: 245, 10575: 268}

In [ ]:
# generate the predicted pairs on the ground truth
predicted_pair = {}
predicted_pair_list = []
for health_id in gt_dict.keys():
    for pair in valid_prediction_pair_multi:
        if str(health_id) in pair:
            predicted_pair_list.append((int(pair[0]), int(pair[1])))
            if health_id in predicted_pair:
                predicted_pair[health_id].append(int(pair[1]))
            else:
                predicted_pair[health_id] = [int(pair[1])]


In [ ]:
r1 = ds1.get_record('3324')
r2 = ds2.get_record('172')
# type(la_df['ID'][0])
# type(rest_df['restaurant_id'][0])
r1,r2

(<__main__.HeathData at 0x7ff7195367f0>, <__main__.RestData at 0x7ff7180833d0>)

In [ ]:
gt = rltk.GroundTruth()

for id1, id2 in gt_dict.items():
    r1 = ds1.get_record(str(id1))
    r2 = ds2.get_record(str(id2))
    gt.add_positive(r1.raw_object['ID'], r2.raw_object['restaurant_id'])

trial = rltk.Trial(gt)
candidate_pairs = rltk.get_record_pairs(ds1, ds2, ground_truth=gt)
for r1, r2 in candidate_pairs:
    result, confidence = rule_based_method(r1, r2)
    trial.add_result(r1, r2, result, confidence)

trial.evaluate()
print('Trial statistics based on Ground-Truth from development set data:')
print(f'tp: {trial.true_positives:.06f} [{len(trial.true_positives_list)}]')
print(f'fp: {trial.false_positives:.06f} [{len(trial.false_positives_list)}]')
print(f'tn: {trial.true_negatives:.06f} [{len(trial.true_negatives_list)}]')
print(f'fn: {trial.false_negatives:.06f} [{len(trial.false_negatives_list)}]')

precision_gt_1 = trial.precision
recall_gt_1 = trial.recall
fscore_gt_1 = trial.f_measure
print('The precision of the ground truth=%.6f'%precision_gt_1)
print('The recall of the ground truth=%.6f'%recall_gt_1)
print('The f1 score of the ground truth=%.6f'%fscore_gt_1)

Trial statistics based on Ground-Truth from development set data:
tp: 0.918367 [45]
fp: 0.000000 [0]
tn: 0.000000 [0]
fn: 0.081633 [4]
The precision of the ground truth=1.000000
The recall of the ground truth=0.918367
The f1 score of the ground truth=0.957447


In [ ]:
# # len(predicted_pair_list)
# predicted_pair_list[:10]
# # predicted_pair

In [ ]:
# # precision = TP / (TP + FP)
# # actually true / predicted true
# tp = 0
# for pair in predicted_pair_list:
#     if pair in gt_dict.items():
#         tp += 1
#     # else:
#     #     print(pair)

# precision = tp / len(predicted_pair_list)
# print(f"precision = {precision}")


In [ ]:
# # recall = TP / (TP + FN)
# # how many true we predicted / number of actual true
# tp = 0
# for pair in gt_dict.items():
#     if pair in predicted_pair_list:
#         tp += 1
#     else:
#         print(pair)

# recall = tp / gt_length
# print(f"recall = {recall}")

In [ ]:
# check the undetected gt
# id1 = 'FA0030157'
# id2 = '1823'
# r1 = ds1.get_record(id1)
# r2 = ds2.get_record(id2)
# result, confidence = rule_based_method(r1, r2)
# result, confidence

Take a look at the pairs we manually defined but not detected.

In [ ]:
la_df[la_df['ID']==5318]

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID
5318,INT HOUSE OF PANCAKES,2912 S SEPULVEDA BLVD,LOS ANGELES,FA0030157,CA,90064,OW0003220,RESTAURANT (61-150) SEATS HIGH RISK,93.294118,"F040,F044,F053,F025,F035,F044,F040,F035,F007,F...","# 40. Plumbing: Plumbing in good repair, prope...",restaurant,61-150,HIGH,5318


In [ ]:
rest_df[rest_df['restaurant_id']==1823]

,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities,open_hours_week,new_address,state,zipcode,city
1823,1823,https://www.yelp.com/biz/ihop-los-angeles-2?os...,IHOP,"2912 S Sepulveda Blvd Los Angeles, CA 90064",2.5,254 reviews,$$,"Breakfast & Brunch, American (Traditional), Bu...",Mon\nOpen 24 hours\nTue\nOpen 24 hours\nWed\nO...,Amenities and More\nOffers Delivery\nOffers Ta...,0.0,2912 S Sepulveda Blvd Los Angeles,CA,90064,Los Angeles


In [ ]:
# f1_score = 2 * precision * recall / (precision + recall)
# print(f"f1 score = {f1_score}")

### Generate the final combined dataset

In [ ]:
linked_df = pd.read_csv('linked_restaurant_predictions_multi.csv')
linked_df.head()

,health.ID,restaurant.ID
0,7507,3362
1,9887,3373
2,9887,3362
3,9887,3344
4,11436,3362


### Problem: same id--multiple restaurants

In [ ]:
linked_dict = dict(zip(linked_df['health.ID'].tolist(), linked_df['restaurant.ID'].tolist()))
# linked_dict

In [ ]:
res_health_df = la_df.loc[la_df['ID'].isin(linked_dict.keys())]
res_health_df.head(2)

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID
38,25 DEGREES,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0003187,CA,90028,OW0030706,RESTAURANT (31-60) SEATS HIGH RISK,92.131579,"F036,F030,F044,F026,F037,F027,F006,F004,F029,F...","# 36. Equipment, utensils and linens: storage ...",restaurant,31-60,HIGH,38
116,7-ELEVEN,380 WORLD WAY T85,LOS ANGELES,FA0172207,CA,90045-5800,OW0101584,"FOOD MKT RETAIL (25-1,999 SF) MODERATE RISK",85.294118,"F035,F036,F030,F054,F007,F043,F033,F053,F044,F...",# 35. Equipment/Utensils - approved; installed...,food market,"25-1,999 SF",MODER,116


In [ ]:
len(res_health_df)

1109

In [ ]:
# res_health_df = res_health_df.drop_duplicates(subset=['FACILITY_NAME'], keep='last') # only keep unique name here
# res_health_df = res_health_df.reset_index(drop=True)
# res_health_df.head()

In [ ]:
res_rest_df = rest_df[rest_df['restaurant_id'].isin(linked_dict.values())]
res_rest_df.head(2)

,url,name,address,score,review_counts,price,category,open_time,amenities,restaurant_id,new_address,state,zipcode,city
2,https://www.yelp.com/biz/wood-los-angeles-8,Wood,"2861 Sunset Blvd Los Angeles, CA 90026",4.0,1017 reviews,$$,"Pizza, Italian, Wine Bars",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nTakes Reservations\nOffers...,2,2861 Sunset Blvd Los Angeles,CA,90026,Los Angeles
3,https://www.yelp.com/biz/jail-joa-los-angeles,Jail Joa,"2500 W 8th St Ste 180 Los Angeles, CA 90057",4.6,(135 reviews),$$,"Gastropubs, Korean, Asian Fusion",Mon\n6:00 PM - 2:00 AM (Next day)\nTue\n6:00 P...,Amenities and More\nOffers Delivery\nOffers Ta...,3,2500 W 8th St Ste 180 Los Angeles,CA,90057,Los Angeles


In [ ]:
len(res_rest_df)

471

In [ ]:
res_health_df['restaurant_id'] = res_health_df["ID"].apply(lambda x: linked_dict[x])
res_health_df.head()

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID,restaurant_id
38,25 DEGREES,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0003187,CA,90028,OW0030706,RESTAURANT (31-60) SEATS HIGH RISK,92.131579,"F036,F030,F044,F026,F037,F027,F006,F004,F029,F...","# 36. Equipment, utensils and linens: storage ...",restaurant,31-60,HIGH,38,3300
116,7-ELEVEN,380 WORLD WAY T85,LOS ANGELES,FA0172207,CA,90045-5800,OW0101584,"FOOD MKT RETAIL (25-1,999 SF) MODERATE RISK",85.294118,"F035,F036,F030,F054,F007,F043,F033,F053,F044,F...",# 35. Equipment/Utensils - approved; installed...,food market,"25-1,999 SF",MODER,116,2297
224,8 ESPRESSOS,11901 SANTA MONICA BLVD STE #112,LOS ANGELES,FA0004207,CA,90025,OW0029730,RESTAURANT (0-30) SEATS LOW RISK,92.600000,"F034,F040,F021,F009,F007,F035,F021,F033,F035,F044","# 34. Warewashing facilities: Adequate, mainta...",restaurant,0-30,LOW,224,2267
231,800 DEGREES PIZZERIA,380 WORLD WAY T31,LOS ANGELES,FA0138164,CA,90045,OW0016382,RESTAURANT (31-60) SEATS HIGH RISK,89.962963,"F037,F040,F053,F044,F035,F033,F044,F049,F009,F...",# 37. Adequate ventilation and lighting; desig...,restaurant,31-60,HIGH,231,2524
246,911 FOOD MARKET,11909 W PICO BLVD,LOS ANGELES,FA0004233,CA,90064,OW0017321,"FOOD MKT RETAIL (25-1,999 SF) LOW RISK",97.500000,"F044,F044,F033,F042","# 44. Floors, walls and ceilings: properly bui...",food market,"25-1,999 SF",LOW,246,2270


In [ ]:
df_inner = pd.merge(res_health_df, res_rest_df, on='restaurant_id', how='inner')
df_inner

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities,new_address,state,zipcode,city
0,25 DEGREES,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0003187,CA,90028,OW0030706,RESTAURANT (31-60) SEATS HIGH RISK,92.131579,"F036,F030,F044,F026,F037,F027,F006,F004,F029,F...","# 36. Equipment, utensils and linens: storage ...",restaurant,31-60,HIGH,38,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
1,LIBRARY BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0035108,CA,90028,OW0030706,RESTAURANT (0-30) SEATS LOW RISK,96.875000,"F033,F040,F033,F044,F014,F033,F044,F036",# 33. Nonfood-contact surfaces clean and in go...,restaurant,0-30,LOW,6797,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
2,PUBLIC,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0017622,CA,90028,OW0030706,RESTAURANT (61-150) SEATS HIGH RISK,92.611111,"F044,F037,F029,F043,F006,F036,F038,F033,F006,F...","# 44. Floors, walls and ceilings: properly bui...",restaurant,61-150,HIGH,9433,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
3,TEDDY'S BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0071995,CA,90028,OW0030706,RESTAURANT (151 + ) SEATS LOW RISK,96.444444,"F036,F035,F006,F033,F036,F040,F044,F034,F033","# 36. Equipment, utensils and linens: storage ...",restaurant,151 +,LOW,11560,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
4,TROPICANA BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0038741,CA,90028,OW0030706,RESTAURANT (0-30) SEATS LOW RISK,94.666667,"F036,F044,F044,F014,F033,F006,F023,F044,F033","# 36. Equipment, utensils and linens: storage ...",restaurant,0-30,LOW,12359,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,YAMASHIRO,1999 N SYCAMORE AVE,LOS ANGELES,FA0240842,CA,90068,OW0236865,RESTAURANT (151 + ) SEATS LOW RISK,94.142857,"F006,F036,F044,F014,F033,F023,F033",# 06. Adequate handwashing facilities supplied...,restaurant,151 +,LOW,13090,219,https://www.yelp.com/biz/yamashiro-hollywood-h...,Yamashiro Hollywood,"1999 N Sycamore Ave Hollywood, CA 90068",3.5,5258 reviews,$$$,"Japanese, Asian Fusion, American (New)",Mon\n5:00 PM - 11:00 PM\nTue\n5:00 PM - 11:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...,1999 N Sycamore Ave Hollywood,CA,90068,Los Angeles
1105,YAMASHIRO MAIN/BANQUET BARS,1999 N SYCAMORE AVE,LOS ANGELES,FA0041590,CA,90068,OW0013408,RESTAURANT (0-30) SEATS MODERATE RISK,97.000000,"F014,F040,F033,F006",# 14. Food contact surface

### can we find all the cities in LA county and remove Santa Monica in new_address -- solved by use zipcode to map

In [ ]:
df_inner[df_inner['restaurant_id']==1080]

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities,new_address,state,zipcode,city


In [ ]:
df_inner[['ID', 'FACILITY_NAME','FACILITY_ADDRESS','FACILITY_CITY', 'SCORE','FACILITY_ZIP','restaurant_id','name','new_address','zipcode','city']]

,ID,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,SCORE,FACILITY_ZIP,restaurant_id,name,new_address,zipcode,city
0,38,25 DEGREES,7000 HOLLYWOOD BLVD,LOS ANGELES,92.131579,90028,3300,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,90028,Los Angeles
1,6797,LIBRARY BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,96.875000,90028,3300,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,90028,Los Angeles
2,9433,PUBLIC,7000 HOLLYWOOD BLVD,LOS ANGELES,92.611111,90028,3300,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,90028,Los Angeles
3,11560,TEDDY'S BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,96.444444,90028,3300,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,90028,Los Angeles
4,12359,TROPICANA BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,94.666667,90028,3300,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,90028,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...
1104,13090,YAMASHIRO,1999 N SYCAMORE AVE,LOS ANGELES,94.142857,90068,219,Yamashiro Hollywood,1999 N Sycamore Ave Hollywood,90068,Los Angeles
1105,13091,YAMASHIRO MAIN/BANQUET BARS,1999 N SYCAMORE AVE,LOS ANGELES,97.000000,90068,219,Yamashiro Hollywood,1999 N Sycamore Ave Hollywood,90068,Los Angeles
1106,13097,YANG CHOW RESTAURANT,819 N BROADWAY,LOS ANGELES,90.716981,90012,89,Yang Chow,819 N Broadway Los Angeles,90012,Los Angeles
1107,13193,YOSHINOYA #1253,1802 E SLAUSON AVE,LOS ANGELES,95.125000,90058,2849,Yoshinoya,1802 E Slauson Ave Los Angeles,90058,Los Angeles


In [ ]:
df_inner.SCORE.describe()

count    1109.000000
mean       92.982006
std         3.304544
min        78.024390
25%        91.176471
50%        93.057143
75%        95.125000
max       100.000000
Name: SCORE, dtype: float64

In [ ]:
df_inner.city.unique()

array(['Los Angeles', 'West Hollywood', 'Culver City', 'Venice',
       'Playa Vista', 'Playa Del Rey', 'Studio City', 'Marina Del Rey'],
      dtype=object)

In [23]:
zipcode_df = pd.read_csv('/content/drive/MyDrive/DSCI 560 Project/Data/zipcode.csv')
# zipcode_df
zipcode_dict = dict(zip([str(x) for x in zipcode_df['zipcode'].values], zipcode_df['city'].values))
# zipcode_dict
df_inner['city'] = df_inner['zipcode'].apply(lambda x:x[:6]).map(zipcode_dict)

In [ ]:
# df_inner.city.isnull().sum()
df_inner.city.unique()

array(['Los Angeles', 'Venice', 'West Hollywood', 'Culver City',
       'Playa Vista'], dtype=object)

Save data

In [ ]:
df_inner.to_csv('combined_rest_health.csv', index=False)

In [ ]:
df_inner

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities,new_address,state,zipcode,city
0,25 DEGREES,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0003187,CA,90028,OW0030706,RESTAURANT (31-60) SEATS HIGH RISK,92.131579,"F036,F030,F044,F026,F037,F027,F006,F004,F029,F...","# 36. Equipment, utensils and linens: storage ...",restaurant,31-60,HIGH,38,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
1,LIBRARY BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0035108,CA,90028,OW0030706,RESTAURANT (0-30) SEATS LOW RISK,96.875000,"F033,F040,F033,F044,F014,F033,F044,F036",# 33. Nonfood-contact surfaces clean and in go...,restaurant,0-30,LOW,6797,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
2,PUBLIC,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0017622,CA,90028,OW0030706,RESTAURANT (61-150) SEATS HIGH RISK,92.611111,"F044,F037,F029,F043,F006,F036,F038,F033,F006,F...","# 44. Floors, walls and ceilings: properly bui...",restaurant,61-150,HIGH,9433,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
3,TEDDY'S BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0071995,CA,90028,OW0030706,RESTAURANT (151 + ) SEATS LOW RISK,96.444444,"F036,F035,F006,F033,F036,F040,F044,F034,F033","# 36. Equipment, utensils and linens: storage ...",restaurant,151 +,LOW,11560,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
4,TROPICANA BAR,7000 HOLLYWOOD BLVD,LOS ANGELES,FA0038741,CA,90028,OW0030706,RESTAURANT (0-30) SEATS LOW RISK,94.666667,"F036,F044,F044,F014,F033,F006,F023,F044,F033","# 36. Equipment, utensils and linens: storage ...",restaurant,0-30,LOW,12359,3300,https://www.yelp.com/biz/the-bridge-los-angeles-2,The Bridge,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,3.6,(7 reviews),$$$,American (New),Mon\n5:30 PM - 10:00 PM\nTue\n5:30 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,7000 Hollywood Blvd Hollywood Roosevelt Hotel ...,CA,90028,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,YAMASHIRO,1999 N SYCAMORE AVE,LOS ANGELES,FA0240842,CA,90068,OW0236865,RESTAURANT (151 + ) SEATS LOW RISK,94.142857,"F006,F036,F044,F014,F033,F023,F033",# 06. Adequate handwashing facilities supplied...,restaurant,151 +,LOW,13090,219,https://www.yelp.com/biz/yamashiro-hollywood-h...,Yamashiro Hollywood,"1999 N Sycamore Ave Hollywood, CA 90068",3.5,5258 reviews,$$$,"Japanese, Asian Fusion, American (New)",Mon\n5:00 PM - 11:00 PM\nTue\n5:00 PM - 11:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...,1999 N Sycamore Ave Hollywood,CA,90068,Los Angeles
1105,YAMASHIRO MAIN/BANQUET BARS,1999 N SYCAMORE AVE,LOS ANGELES,FA0041590,CA,90068,OW0013408,RESTAURANT (0-30) SEATS MODERATE RISK,97.000000,"F014,F040,F033,F006",# 14. Food contact surface

In [18]:
import pandas as pd

In [29]:
df = pd.read_csv('/content/drive/MyDrive/DSCI 560 Project/Data/Public_Health_LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_INSPECTIONS_.csv')
df.head()

,ACTIVITY_DATE,OWNER_ID,OWNER_NAME,FACILITY_ID,FACILITY_NAME,RECORD_ID,PROGRAM_NAME,PROGRAM_STATUS,PROGRAM_ELEMENT__PE_,PE_DESCRIPTION,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_STATE,FACILITY_ZIP,SERVICE_CODE,SERVICE_DESCRIPTION,SCORE,SERIAL_NUMBER,EMPLOYEE_ID,ObjectId
0,2017/05/23 00:00:00+00,OW0021044,LEE CHEA,FA0025785,MR K'S DONUTS,PR0037572,MR K'S DONUTS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,4831 N PARAMOUNT BLVD,LAKEWOOD,CA,90712,1,ROUTINE INSPECTION,94,DAOQJGANB,EE0000606,1
1,2017/06/05 00:00:00+00,OW0038621,WEST END PUB LLC.,FA0055840,WEST END,PR0002073,WEST END,ACTIVE,1633,RESTAURANT (31-60) SEATS LOW RISK,1301 5TH ST,SANTA MONICA,CA,90401,1,ROUTINE INSPECTION,94,DAY80PGWA,EE0000997,2
2,2017/06/06 00:00:00+00,OW0012488,FUNG KIN,FA0014141,CHANG LONG GINSENG CO.,PR0040250,CHANG LONG GINSENG CO.,ACTIVE,1610,"FOOD MKT RETAIL (1-1,999 SF) LOW RISK",18924 E GALE AVE,ROWLAND HEIGHTS,CA,91748,1,ROUTINE INSPECTION,96,DAEMQKZPY,EE0000263,3
3,2017/06/06 00:00:00+00,OW0011460,EZ 02 LLC,FA0012801,EL ZOCALO,PR0012306,EL ZOCALO,ACTIVE,1635,RESTAURANT (31-60) SEATS HIGH RISK,900 W ANAHEIM ST,WILMINGTON,CA,90744,1,ROUTINE INSPECTION,90,DAYYBKWRW,EE0000486,4
4,2017/06/06 00:00:00+00,OW0101299,HKLEE CORPORATION,FA0027282,KOTE DAEJI,PR0017132,KOTE DAEJI,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,14816 S BEACH BLVD,LA MIRADA,CA,90638,1,ROUTINE INSPECTION,90,DASUBSKNQ,EE0000182,5


In [30]:
zipcode_df = pd.read_csv('/content/drive/MyDrive/DSCI 560 Project/Data/zipcode.csv')
# zipcode_df
zipcode_dict = dict(zip([str(x) for x in zipcode_df['zipcode'].values], zipcode_df['city'].values))

In [34]:
df['city'] = df['FACILITY_ZIP'].astype(str).apply(lambda x:x[:6]).map(zipcode_dict)

In [64]:
df[df['FACILITY_CITY']=='PASADENA'].sort_values(by='SCORE').head(10)

,ACTIVITY_DATE,OWNER_ID,OWNER_NAME,FACILITY_ID,FACILITY_NAME,RECORD_ID,PROGRAM_NAME,PROGRAM_STATUS,PROGRAM_ELEMENT__PE_,PE_DESCRIPTION,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_STATE,FACILITY_ZIP,SERVICE_CODE,SERVICE_DESCRIPTION,SCORE,SERIAL_NUMBER,EMPLOYEE_ID,ObjectId,city
249115,2020/06/16 00:00:00+00,OW0258316,PLK NV INC,FA0267799,POPEYES LOUSIANA KITCHEN #12984,PR0219470,POPEYES LOUSIANA KITCHEN #12984,ACTIVE,1634,RESTAURANT (31-60) SEATS MODERATE RISK,311 S ROSEMEAD BLVD,PASADENA,CA,91107,1,ROUTINE INSPECTION,76,DANWMPQ31,EE0000964,249116,Pasadena
91691,2018/05/23 00:00:00+00,OW0023105,MAMMA'S BRICK OVEN INC.,FA0038187,MAMMA'S BRICK OVEN PIZZA,PR0001353,MAMMA'S BRICK OVEN PIZZA,INACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,311 S ROSEMEAD BLVD,PASADENA,CA,91107,1,ROUTINE INSPECTION,80,DA5WGDG6G,EE0000886,91692,Pasadena
144007,2019/02/12 00:00:00+00,OW0036674,"TUFENKJI, TONY NAZARETH",FA0037987,TNT C-STORE,PR0047851,TNT C-STORE,INACTIVE,1610,"FOOD MKT RETAIL (1-1,999 SF) LOW RISK",3880 E COLORADO BLVD,PASADENA,CA,91107,401,OWNER INITIATED INSPECTION,81,DAAILVBAE,EE0000886,144008,Pasadena
299084,2021/08/10 00:00:00+00,OW0019058,KIM FAMILY INC,FA0015000,GIN SUSHI JAPANESE RESTAURANT,PR0010491,GIN SUSHI JAPANESE RESTAURANT,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,3589 E COLORADO BLVD,PASADENA,CA,91107,1,ROUTINE INSPECTION,81,DAJD0NL7S,EE0000964,299085,Pasadena
201240,2019/10/09 00:00:00+00,OW0245960,EATON ALTADENA GOLF LLC,FA0252308,EATON CANYON GOLF COURSE,PR0201564,EATON CANYON GOLF COURSE,ACTIVE,1634,RESTAURANT (31-60) SEATS MODERATE RISK,1150 SIERRA MADRE VILLA AVE,PASADENA,CA,91107,1,ROUTINE INSPECTION,81,DATNQFLM7,EE0001124,201241,Pasadena
259006,2020/08/25 00:00:00+00,OW0004884,B-MANS RESTAURANTS INC,FA0010124,B-MAN'S TERIYAKI & BURGERS,PR0008759,B-MAN'S TERIYAKI & BURGERS,ACTIVE,1635,RESTAURANT (31-60) SEATS HIGH RISK,3007 E HUNTINGTON DR STE 102,PASADENA,CA,91107,1,ROUTINE INSPECTION,81,DAWUTY8ZC,EE0001166,259007,Pasadena
183526,2019/05/30 00:00:00+00,OW0019058,KIM FAMILY INC,FA0015000,GIN SUSHI JAPANESE RESTAURANT,PR0010491,GIN SUSHI JAPANESE RESTAURANT,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,3589 E COLORADO BLVD,PASADENA,CA,91107,1,ROUTINE INSPECTION,82,DAP05BO4W,EE0000886,183527,Pasadena
224556,2020/02/20 00:00:00+00,OW0010270,DON FRANCO ENTERPRISES,FA0041724,NIKKIC'S,PR0020842,NIKKIC'S,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,470 S ROSEMEAD BLVD,PASADENA,CA,91107,1,ROUTINE INSPECTION,83,DAOGBYBRO,EE0000964,224557,Pasadena
62303,2018/02/06 00:00:00+00,OW0012163,FORESIGHT INVESTMENT GRP INC,FA0055645,YANG CHOW RESTAURANT,PR0015153,YANG CHOW RESTAURANT,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,3777 E COLORADO BLVD,PASADENA,CA,91107,1,ROUTINE INSPECTION,83,DAURD8A0M,EE0000886,62304,Pasadena
233905,2020/02/04 00:00:00+00,OW0028996,PRESIDENT THAI CORP,FA0029059,PRESIDENT THAI RESTAURANT,PR0014812,PRESIDENT THAI RESTAURANT,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,498 S ROSEMEAD BLVD,PASADENA,CA,91107,1,ROUTINE INSPECTION,84,DALSM0HVH,EE0000964,233906,Pasadena


In [ ]:
[['FACILITY_NAME', 'name', 'FACILITY_ZIP', 'review_counts', 'price', 'category', 'type', 
           'size', 'open_hours_week', 'SCORE', 'score', 'num_photos', 'is_bus_web', 'is_phone_number', 'is_message_bus',
           'num_attributes', 'num_questions', 'comments_list']]

In [66]:
comment1 = \
"""
1. Parking & Restrooms - shared parking with other stores, but at least no fee nor limit, gender neutral restrooms available for customers
2. Contactless payment- yes, apple pay accepted
3. Reservations/Appointment - first come first serve
4. Customer Service/Assistance - all employees have been nice
5. Food/Health Grade/Cleanness- love coming here especially in the summer, mango slushy is my favorite, didn't see the health grade or at least it didn't stand out to me, clean & well kept no trash on the floor
"""

comment2 = """I found the place by accident.  I don't generally eat Mexican food because it just isn't on my RADAR.  I lived with an Italian cook for over 15 years who was quite versatile in the kitchen, but occasionally needed or wanted a quick fix that was high in protein and easy to get down.  Burritos, taquitos, tacos... meats and vegetables mixed with a gravy called sofrito were sometimes all he needed.
El Super Burrito is on the path of my nightly walks.  Imagine how odd it sounds to go on a healthy walk and stop by a burrito stand!
It is a cash only establishment and there is a working cash machine in a small, covered area that has picnic tables.  I have seen workers and families eat here and everyone seems happy.  The window also does an enormous amount of carry out and it advertises its service for catering.
I pretty much order the same thing every time I am there.  Two tongue tacos please.  They each come in two-ply, soft, corn tortillas.  I ask them told hold the salsa and cilantro (they wind up giving it to me in a cup) and they add hot fresh corn chips on the side.
The food is very good and the service is friendly.  I can order in English or in Spanish and the responses are appropriate and clear.  The pricing is competitive and fair (come on, have you seen how much eggs cost in a store?).  I've also had huevos rancheros and a bean and cheese burrito.  I've never been disappointed.
As I had mentioned, there is space to eat out as well as an option take away the meal.  Considering the number of Mexican restaurants in the neighborhood (including the fast food shops such as Taco Bell or Del Taco), I believe that El Super Burrito stands out as an excellent food value for your dollar."""

comment3 = """It use to be my favorite, now it's making a comeback. I love coming here.
The food is always great and the service was good."""

comment4 = """My "home" course of sorts. It's a nine-hole course that offers some par 4s and even an uphill par 5. It's in decent shape most of the time. Yes, as others have stated, the fairways are patchy and the cart paths have a few potholes. But the greens are surprisingly well-maintained and in great shape.

The rates are decent, even with a cart. But they usually run out of carts as this is a popular course due to the price point and ease of booking. You can walk-on, no hassle, and not feel intimidated. This is a public course that anyone can play and have fun on.

My only complaint is that the range is meh...the balls are well beaten and in really bad shape. It's pricey to hit a bucket of old balls. Also, no night lights for the range. That's too bad."""

comment5 = """I'm updating my review because the owner has made a great effort to rectify this customers satisfaction and that goes a long way! Food is consistently good here.

Tried to call in a phone order for pick up but the girl on the phone said they don't take phone orders that they only accept orders online. So in an effort to save time I order online before I head over. When I get there I asked the cashier if my order is ready and he tells me they are still working on it but they will call my name when it is ready. So I'm sitting and waiting and I'm seeing other people who order after me are getting their orders so I go up and ask them again if my order is ready and apparently it was sitting on the counter for the last 15 minutes. No one called my name to notify me my order was ready and I ordered online and effort to save time. Very displeased with the service I received. My family and I frequent here but this last time was very disappointing
"""

In [67]:
pasadena_case = pd.DataFrame({
    "FACILITY_NAME":['HALF & HALF TEA HOUSE','EL SUPER BURRITO MEXICAN FAST FOOD RESTAURANT','GIN SUSHI JAPANESE RESTAURANT',\
                     'EATON CANYON GOLF COURSE',"B-MAN'S TERIYAKI & BURGERS"],
    'name':['Half and Half Tea Express','El Super Burrito','Gin Sushi','Eaton Canyon Golf Course',"B-Man's Teriyaki & Burgers"],
    'FACILITY_ZIP':['91107', '91107','91107','91107','91107'],
    'review_counts':['781','307','1892','72','1040'],
    'price':['$$','$$',"$$",None,'$$'], 
    'category':['Coffee & Tea, Juice Bars & Smoothies','Mexican','Sushi Bars, Japanese, Seafood','Golf','Burgers, Fast Food, Japanese'], 
    'type':['restaurant','restaurant','restaurant','restaurant','restaurant'], 
    'size':['0-30','0-30','61-150','31-60','31-60'],
    'open_hours_week':[49.5, 97,70,91,70], 
    'SCORE':[97,97,81,81,81], 
    'score':[3.5,4,3.5,3,4], 
    'num_photos':['See all 939 photos','See all 295 photos','See all 1797 photos','See all 149 photos','See all 717 photos'], 
    'is_bus_web':[True, False, True,True,True], 
    'is_phone_number':[True,True,True,True,True], 
    'is_message_bus':[False,False,False,False,False],
    'num_attributes':[11,21,34, None,26], 
    'num_questions':['See all 4 questions',None,'See all 7 questions','See all 4 questions','See all 2 questions'], 
    'comments_list':[comment1,comment2,comment3,comment4,comment5]
})

In [48]:
pasadena_case.FACILITY_ZIP[0]

'91107'

In [68]:
pasadena_case.to_csv('use_case.csv', index=False, header=True)